# Building and Training our own ResNet34 model

Follow this paper https://arxiv.org/pdf/1908.04913v1.pdf, we decide to use the UTK face dataset and ResNet34 to build our own model for thie project. Below is the steps to build the model.

In [1]:
!pip install imutils

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.6/install/bin/python3.8 -m pip install --upgrade pip' command.


In [31]:
import tensorflow as tf
import numpy as np

# Definition of the identity block of the ResNet34
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x


# Definition of Convolutional Block of the ResNet34
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x


# Definition of the ResNet34
def ResNet34(shape = (200, 200, 3), classes = 4):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

## Age prediction
### Preprocessing
    We use the dataset UTKFace to train our database, the data set contains totally 23708 images, we randomly select 2708 of them as validation set, 500 of them as test set. The name of each image already contains the label info of the image: the format of the label is [age]_[gender]_[race]_[date&time].jpg.
    In the preprocessing, we do:
    1. Read the image and turn them into rgb.
    2. Shuffle the train set.
    3. Normalize the train set and test set, by dividing each pixil value by 255, where 255 is the max value of each pixel.
    

In [3]:
import random
import cv2

from imutils import paths


image_original_folder = "../resource/UTKFace_filtered/"
image_paths = list(paths.list_images(image_original_folder))

test_set_size = 100 # There are total 23708 images in UTKFace, so this should not exceed 23708
non_training_data = random.sample(range(0, 10446), test_set_size)
test_indexes = set(non_training_data)

train_set = []
train_labels = []
test_set = []
test_labels = []
for (i, imagePath) in enumerate(image_paths):
    if i in test_indexes:
        added_set = test_set
        added_labels = test_labels
    else:
        added_set = train_set
        added_labels = train_labels
    # the format of the label is [age]_[gender]_[race]_[date&time].jpg,
    # here only age and race are needed
    labels = imagePath.split("/")[-1].split(".")[0].split("_")
    if len(labels) != 4:
        # few images in the UTKFace have broken labels, ignore them
        continue
    labels = [int(labels[0])]
    label = 0 if labels[0] < 21 else 1
    
    image = cv2.imread(imagePath)
    added_set.append(image)
    added_labels.append(label)

print(len(train_set))
print(len(train_labels))
print(len(test_set))
print(len(test_labels))




10346
10346
100
100


In [4]:
# shuffle the train data
train_XY = list(zip(train_set, train_labels))
random.shuffle(train_XY)

train_set, train_labels = zip(*train_XY)
train_set = list(train_set)
train_labels = list(train_labels)


In [5]:
# normalize the data image
train_set = np.array(train_set)
train_labels = np.array(train_labels)
test_set = np.array(test_set)
test_labels = np.array(test_labels)
normed_train_set = train_set/255
normed_test_set = test_set/255

### Train the model
    Besides training the model, we also use the test set to see the performance of the model

In [6]:
num_epochs = 32
learning_rate = 0.001
batch_size = 32
model = ResNet34(shape = (200, 200, 3) ,classes=2)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
                    initial_learning_rate=learning_rate, decay_steps=num_epochs*100, decay_rate=0.9)), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
model.fit(normed_train_set, train_labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/32
324/324 [==============================] - 35s 108ms/step - loss: 0.6358 - accuracy: 0.6468
Epoch 2/32
324/324 [==============================] - 34s 104ms/step - loss: 0.5514 - accuracy: 0.7510
Epoch 3/32
324/324 [==============================] - 34s 104ms/step - loss: 0.5140 - accuracy: 0.7881
Epoch 4/32
324/324 [==============================] - 34s 104ms/step - loss: 0.4879 - accuracy: 0.8187
Epoch 5/32
324/324 [==============================] - 33s 103ms/step - loss: 0.4708 - accuracy: 0.8366
Epoch 6/32
324/324 [==============================] - 33s 103ms/step - loss: 0.4525 - accuracy: 0.8579
Epoch 7/32
324/324 [==============================] - 33s 103ms/step - loss: 0.4378 - accuracy: 0.8780
Epoch 8/32
324/324 [==============================] - 34s 104ms/step - loss: 0.4267 - accuracy: 0.8897
Epoch 9/32
324/324 [==============================] - 33s 103ms/step - loss: 0.4121 - accuracy: 0.9047
Epoch 10/32
324/324 [==============================] - 33s 103ms/step - l

In [7]:
model.save("2021_12_6_11_28_age")
model = tf.keras.models.load_model("2021_12_6_11_28_age")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: 2021_12_6_11_28_age/assets


In [10]:
from sklearn import metrics
from sklearn.metrics import classification_report

prediction_prob = model.predict(normed_test_set)

In [11]:
results = tf.argmax(prediction_prob, axis=1)
metrics.accuracy_score(test_labels, results)
print(classification_report(y_true=test_labels,y_pred=results))

              precision    recall  f1-score   support

           0       0.89      0.92      0.91        52
           1       0.91      0.88      0.89        48

    accuracy                           0.90       100
   macro avg       0.90      0.90      0.90       100
weighted avg       0.90      0.90      0.90       100



### Predict on the twitter user profile
    Below we will run the age prediction on the first batch of the twitter user profile image.
    The preprocess steps are same as what have been down to the training set.

In [12]:
profile0_folder = "../resource/cropped_profile/"
image_paths = list(paths.list_images(profile0_folder))

test_set0 = []
test_labels0 = []
for (i, imagePath) in enumerate(image_paths):
    # the format of the label is [age]_[race]_[id].jpeg,
    # here only age and race are needed
    labels = imagePath.split("/")[-1].split(".")[0].split("_")
    if len(labels) != 3:
        # few images in the UTKFace have broken labels, ignore them
        continue
    labels = [int(labels[0])]
    label = 0 if labels[0] < 21 else 1
    
    image = cv2.imread(imagePath)
    test_set0.append(image)
    test_labels0.append(label)

test_set0 = np.array(test_set0)
test_labels0 = np.array(test_labels0)
normed_test_set0 = test_set0/255

prediction_prob = model.predict(normed_test_set0)
results = tf.argmax(prediction_prob, axis=1)
print(metrics.accuracy_score(test_labels0, results))
print(classification_report(y_true=test_labels0,y_pred=results))



0.7360090446579989
              precision    recall  f1-score   support

           0       0.00      0.14      0.00         7
           1       1.00      0.74      0.85      1762

    accuracy                           0.74      1769
   macro avg       0.50      0.44      0.43      1769
weighted avg       0.99      0.74      0.84      1769



### Predict on the second user profile set


In [16]:
profile1_folder = "../resource/cropped_profile1/"
image_paths = list(paths.list_images(profile1_folder))

test_set1 = []
test_labels1 = []
test_id1 = []
for (i, imagePath) in enumerate(image_paths):
    # the format of the label is [age]_[id].jpeg,
    # here only age and race are needed
    labels = imagePath.split("/")[-1].split(".")[0].split("_")
    if len(labels) != 2:
        # few images in the UTKFace have broken labels, ignore them
        continue
    id = labels[1]
    labels = [int(labels[0])]
    label = 0 if labels[0] < 21 else 1
    
    image = cv2.imread(imagePath)
    test_set1.append(image)
    test_id1.append(id)
    test_labels1.append(label)

test_set1 = np.array(test_set1)
test_labels1 = np.array(test_labels1)
test_id1 = np.array(test_id1)
normed_test_set1 = test_set1/255


prediction_prob = model.predict(normed_test_set1)
results = tf.argmax(prediction_prob, axis=1)
print(metrics.accuracy_score(test_labels1, results))
print(classification_report(y_true=test_labels1,y_pred=results))

0.5560407569141194
              precision    recall  f1-score   support

           0       0.41      0.29      0.34       269
           1       0.61      0.73      0.67       418

    accuracy                           0.56       687
   macro avg       0.51      0.51      0.50       687
weighted avg       0.53      0.56      0.54       687



In [33]:
import pandas as pd

test_id1 = test_id1.reshape(-1, 1)
test_labels1 = test_labels1.reshape(-1, 1)
output_nparray = np.hstack((test_id1, test_labels1, prediction_prob))
output_csv = pd.DataFrame(data=output_nparray, columns=["id", "label", "P0", "P1"])
output_csv.to_csv("age_precition_dataset2.csv", index=False)

In [37]:
profile2_folder = "../resource/cropped_profile2/"
image_paths = list(paths.list_images(profile2_folder))

test_set2 = []
test_labels2 = []
test_id2 = []
for (i, imagePath) in enumerate(image_paths):
    # the format of the label is [age]_[race]_[id].jpeg,
    # here only age and race are needed
    labels = imagePath.split("/")[-1].split(".")[0].split("_")
    if len(labels) != 3:
        # few images in the dataset have broken labels, ignore them
        continue
    id = labels[2]
    labels = [int(labels[0])]
    label = 0 if labels[0] < 21 else 1
    
    image = cv2.imread(imagePath)
    test_set2.append(image)
    test_labels2.append(label)
    test_id2.append(id)

test_set2 = np.array(test_set2)
test_labels2 = np.array(test_labels2)
test_id2 = np.array(test_id2)
normed_test_set2 = test_set2/255

prediction_prob = model.predict(normed_test_set2)
results = tf.argmax(prediction_prob, axis=1)
print(metrics.accuracy_score(test_labels2, results))
print(classification_report(y_true=test_labels2,y_pred=results))

0.6288870703764321
              precision    recall  f1-score   support

           0       0.00      0.30      0.01        10
           1       1.00      0.63      0.77      2434

    accuracy                           0.63      2444
   macro avg       0.50      0.47      0.39      2444
weighted avg       0.99      0.63      0.77      2444



In [38]:
import pandas as pd

test_id2 = test_id2.reshape(-1, 1)
test_labels2 = test_labels2.reshape(-1, 1)
output_nparray = np.hstack((test_id2, test_labels2, prediction_prob))
output_csv = pd.DataFrame(data=output_nparray, columns=["id", "label", "P0", "P1"])
output_csv.to_csv("age_precition_dataset1.csv", index=False)

## Race prediction
### Preprocessing


In [40]:
import random
import cv2

from imutils import paths


image_original_folder = "../resource/UTKFace/"
image_paths = list(paths.list_images(image_original_folder))

test_set_size = 100 # There are total 23708 images in UTKFace, so this should not exceed 23708
non_training_data = random.sample(range(0, 23708), test_set_size)
test_indexes = set(non_training_data)

train_set = []
train_labels = []
test_set = []
test_labels = []
for (i, imagePath) in enumerate(image_paths):
    if i in test_indexes:
        added_set = test_set
        added_labels = test_labels
    else:
        added_set = train_set
        added_labels = train_labels
    # the format of the label is [age]_[gender]_[race]_[date&time].jpg,
    # here only age and race are needed
    labels = imagePath.split("/")[-1].split(".")[0].split("_")
    if len(labels) != 4:
        # few images in the UTKFace have broken labels, ignore them
        continue
    
    # In UTKFace, there are 4 races, but in our twitter data, there are 5 races, to reach an agreement, here we unify the 
    # standard:
    # 0---White, so for twitter dataset, we need to change 4 to 0
    # 1---Black, there is no change needed
    # 2---Asian, for twitter dataset, we need to change 3 to 2
    # 3---Other, including: Latino/Hispanic/Indian and so on. So for twitter dataset, we need to change 2, 5 to 3; for UTKFace
    # dataset, we need to change 4 to 3.
    labels = [int(labels[2])]
    if labels[0] == 4:
        labels[0] = labels[0] - 1
    label = labels[0]
    image = cv2.imread(imagePath)
    added_set.append(image)
    added_labels.append(label)

print(len(train_set))
print(len(train_labels))
print(len(test_set))
print(len(test_labels))




23605
23605
100
100


In [41]:
import numpy as np
# shuffle the train data
train_XY = list(zip(train_set, train_labels))
random.shuffle(train_XY)

train_set, train_labels = zip(*train_XY)
train_set = list(train_set)
train_labels = list(train_labels)

# normalize the data image
train_set = np.array(train_set)
train_labels = np.array(train_labels)
test_set = np.array(test_set)
test_labels = np.array(test_labels)
normed_train_set = train_set/255
normed_test_set = test_set/255

In [43]:
# This part is preprocessing for twitter dataset

profile0_folder = "../resource/cropped_profile/"
image_paths = list(paths.list_images(profile0_folder))

test_set0 = []
test_labels0 = []
test_id0 = []
for (i, imagePath) in enumerate(image_paths):
    # the format of the label is [age]_[race]_[id].jpeg,
    # here only age and race are needed
    labels = imagePath.split("/")[-1].split(".")[0].split("_")
    if len(labels) != 3:
        # few images in the UTKFace have broken labels, ignore them
        continue
    id = labels[2]
    labels = [int(labels[1])]
    if labels[0] == 4:
        labels[0] = 0
    elif labels[0] == 3:
        labels[0] = 2
    elif labels[0] == 2 or labels[0] == 5:
        labels[0] = 3
    label = labels[0]
    image = cv2.imread(imagePath)
    test_set0.append(image)
    test_labels0.append(label)
    test_id0.append(id)

test_set0 = np.array(test_set0)
test_labels0 = np.array(test_labels0)
test_id0 = np.array(test_id0)
normed_test_set0 = test_set0/255

In [44]:
profile2_folder = "../resource/cropped_profile2/"
image_paths = list(paths.list_images(profile2_folder))

test_set2 = []
test_labels2 = []
test_id2 = []
for (i, imagePath) in enumerate(image_paths):
    # the format of the label is [age]_[race]_[id].jpeg,
    # here only age and race are needed
    labels = imagePath.split("/")[-1].split(".")[0].split("_")
    if len(labels) != 3:
        # few images in the dataset have broken labels, ignore them
        continue
    id = labels[2]
    labels = [int(labels[1])]
    if labels[0] == 4:
        labels[0] = 0
    elif labels[0] == 3:
        labels[0] = 2
    elif labels[0] == 2 or labels[0] == 5:
        labels[0] = 3
    label = labels[0]
    image = cv2.imread(imagePath)
    test_set2.append(image)
    test_labels2.append(label)
    test_id2.append(id)

test_set2 = np.array(test_set2)
test_labels2 = np.array(test_labels2)
test_id2 = np.array(test_id2)
normed_test_set2 = test_set2/255


### Train the model

In [47]:
num_epochs = 64
learning_rate = 0.001
batch_size = 32
model = ResNet34(shape = (200, 200, 3) ,classes=4)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
                    initial_learning_rate=learning_rate, decay_steps=num_epochs*1000, decay_rate=0.9)), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
model.fit(normed_train_set, train_labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/64
738/738 [==============================] - 77s 104ms/step - loss: 1.2527 - accuracy: 0.4772
Epoch 2/64
738/738 [==============================] - 77s 104ms/step - loss: 1.1877 - accuracy: 0.5461
Epoch 3/64
738/738 [==============================] - 77s 104ms/step - loss: 1.1369 - accuracy: 0.6014
Epoch 4/64
738/738 [==============================] - 77s 104ms/step - loss: 1.0985 - accuracy: 0.6424
Epoch 5/64
738/738 [==============================] - 77s 104ms/step - loss: 1.0558 - accuracy: 0.6874
Epoch 6/64
738/738 [==============================] - 76s 104ms/step - loss: 1.0133 - accuracy: 0.7339
Epoch 7/64
738/738 [==============================] - 76s 103ms/step - loss: 0.9803 - accuracy: 0.7702
Epoch 8/64
738/738 [==============================] - 77s 104ms/step - loss: 0.9585 - accuracy: 0.7920
Epoch 9/64
738/738 [==============================] - 76s 104ms/step - loss: 0.9352 - accuracy: 0.8186
Epoch 10/64
738/738 [==============================] - 77s 104ms/step - l

In [48]:
from sklearn import metrics
from sklearn.metrics import classification_report

prediction_prob = model.predict(normed_test_set)
results = tf.argmax(prediction_prob, axis=1)
print(metrics.accuracy_score(test_labels, results))
print(classification_report(y_true=test_labels,y_pred=results))

0.76
              precision    recall  f1-score   support

           0       0.78      0.91      0.84        44
           1       0.75      0.62      0.68        24
           2       0.77      0.71      0.74        14
           3       0.69      0.61      0.65        18

    accuracy                           0.76       100
   macro avg       0.75      0.71      0.73       100
weighted avg       0.76      0.76      0.75       100



In [49]:
model.save("2021_12_6_15_31_race")
model = tf.keras.models.load_model("2021_12_6_15_31_race")

INFO:tensorflow:Assets written to: 2021_12_6_15_31_race/assets


### Predict for twitter users

In [50]:
prediction_prob = model.predict(normed_test_set0)
results = tf.argmax(prediction_prob, axis=1)
print(metrics.accuracy_score(test_labels0, results))
print(classification_report(y_true=test_labels0,y_pred=results))

0.604296212549463
              precision    recall  f1-score   support

           0       0.87      0.68      0.76      1423
           1       0.22      0.51      0.31       134
           2       0.11      0.17      0.13        53
           3       0.10      0.18      0.13       159

    accuracy                           0.60      1769
   macro avg       0.33      0.38      0.33      1769
weighted avg       0.73      0.60      0.65      1769



In [51]:
prediction_prob = model.predict(normed_test_set2)
results = tf.argmax(prediction_prob, axis=1)
print(metrics.accuracy_score(test_labels2, results))
print(classification_report(y_true=test_labels2,y_pred=results))

0.5433715220949263
              precision    recall  f1-score   support

           0       0.86      0.60      0.71      1926
           1       0.19      0.55      0.28       218
           2       0.07      0.18      0.10        74
           3       0.15      0.20      0.17       226

    accuracy                           0.54      2444
   macro avg       0.32      0.38      0.32      2444
weighted avg       0.71      0.54      0.60      2444



In [53]:
import pandas as pd

test_id2 = test_id2.reshape(-1, 1)
test_labels2 = test_labels2.reshape(-1, 1)
output_nparray = np.hstack((test_id2, test_labels2, prediction_prob))
output_csv = pd.DataFrame(data=output_nparray, columns=["id", "label", "P0", "P1", "P2", "P3"])
output_csv.to_csv("race_precition_dataset1.csv", index=False)